<a href="https://colab.research.google.com/github/hkbu-kennycheng/comp7240/blob/main/lab1_collaborative_filtering_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab1: Collaborative filtering methods

In [ ]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 51 kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619435 sha256=48f17fdf1544b87086da74a45f5b056668bb6b67dc13e64cce094d0b54faa508
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


## Rating matrix

|Rating|item 1|item 2|item 3|item 4|item 5| ... |
|------|------|------|------|------|------|-----|
|user 1| 5    |      | 4    | 1    |      |     |
|user 2|      | 3    |      | 3    |      |     |
|user 3|      | 4    | 4    | 1    |      |     |
|user 4| 4    | 4    | 5    |      |      |     |
|user 5| 2    | 4    |      | 5    | 2    |     |
|...   |      |      |      |      |      |     |

## Dataset: MovieLens 100K


In [ ]:
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

# Load the movielens-100k dataset
data = Dataset.load_builtin('ml-100k', prompt=False)

# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=.25)

Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


# User-based vs item-based Collaborative Filtering

User-user collaborative filtering is a technique to find similar users based on the ratings they give using a rating matrix.

In [ ]:
# 

## 

### Item based method using Centered K-Nearest Neighbours (k-NN)

In [ ]:
from surprise import KNNWithMeans
from surprise.model_selection import cross_validate

# To use item-based cosine similarity
algo = KNNWithMeans(sim_options={
  "name": "cosine",
  "user_based": False,  # Compute  similarities between items
})

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9406  0.9382  0.9417  0.9480  0.9427  0.9422  0.0033  
MAE (testset)     0.7381  0.7359  0.7419  0.7449  0.7425  0.7407  0.0032  
Fit time          1.61    1.59    1.64    1.57    1.60    1.60    0.02    
Test time         4.38    4.30    4.40    4.41    4.31    4.36    0.05    


{'fit_time': (1.6057422161102295,
  1.5907692909240723,
  1.6442632675170898,
  1.5733733177185059,
  1.5957517623901367),
 'test_mae': array([0.73810342, 0.73591826, 0.74188076, 0.7449242 , 0.74252372]),
 'test_rmse': array([0.94056374, 0.93815336, 0.94168466, 0.94799637, 0.94274849]),
 'test_time': (4.379783391952515,
  4.2970030307769775,
  4.397813320159912,
  4.412890672683716,
  4.313941955566406)}

## User based method using Centered K-Nearest Neighbours (k-NN)

In [ ]:
# To use user-based cosine similarity
algo = KNNWithMeans(sim_options={
    "name": "cosine",
    "user_based": True,  # Compute similarities between users
})

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9497  0.9538  0.9614  0.9548  0.9578  0.9555  0.0039  
MAE (testset)     0.7486  0.7524  0.7591  0.7539  0.7572  0.7542  0.0037  
Fit time          0.95    0.98    0.96    0.98    0.98    0.97    0.01    
Test time         3.95    3.86    3.88    3.91    3.83    3.89    0.04    


{'fit_time': (0.9508118629455566,
  0.9795711040496826,
  0.9599606990814209,
  0.9839956760406494,
  0.9839878082275391),
 'test_mae': array([0.74855623, 0.75238425, 0.75905195, 0.75385061, 0.75720022]),
 'test_rmse': array([0.94970363, 0.9537819 , 0.96143863, 0.954805  , 0.95777646]),
 'test_time': (3.9457554817199707,
  3.8625736236572266,
  3.882866382598877,
  3.908400774002075,
  3.831212043762207)}

# CoClustering


In [ ]:
from surprise import CoClustering

algo = CoClustering()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)